In [17]:
import pandas as pd

# ----------------------------
# Step 1. Read the first file: cooking methods and other info
# ----------------------------
df_method = pd.read_excel("Asia_recipes_processed_ali.xlsx")
df_method.set_index('index', inplace=True)

# Ensure cooking time is numeric (in minutes); default to 30 minutes if conversion fails
df_method['cookingtime'] = pd.to_numeric(df_method['cookingtime'], errors='coerce').fillna(30)

# ----------------------------
# Step 2. Read the serving information file (Asia_recipes_clear.xlsx)
# ----------------------------
df_serving = pd.read_excel("Asia_recipes_clear.xlsx")
df_serving['idrecipe'] = df_serving['idrecipe'].astype(str)
df_method.index = df_method.index.astype(str)

df_serving['servings'] = df_serving['serving_new']
df_serving_subset = df_serving[['idrecipe', 'servings']]

df_method = df_method.reset_index().merge(df_serving_subset, left_on='index', right_on='idrecipe', how='left')
df_method.set_index('index', inplace=True)

# ----------------------------
# Step 3. Country-specific CO2 emission factors for electricity (g CO2/kWh)
# ----------------------------
country_co2_factors = {
    'Japan': 485,
    'China': 581,
    'South Korea': 431,
    'India': 713,
    'Thailand': 549.6,
    'Vietnam': 475.4,
    'Singapore': 470.8,
    'Indonesia': 600,
    'Malaysia': 605.8,
    'Philippines': 640,
    'default': 594,
    "Mongolia": 775.3,
    "Maldives": 611.8
}

def get_kWh_to_co2(country):
    return country_co2_factors.get(country, country_co2_factors['default'])

# ----------------------------
# Step 4. Parameters and GHG emission calculation functions for appliances
# ----------------------------
mj_to_co2 = 13.6 * 44 / 12   # ≈50 g CO2 per MJ
C_w = 4.2  # Specific heat of water (J/g·°C)
K_water_room_temp = 20  # Initial water temperature
H_w = 2250  # Latent heat of vaporization of water (J/g)
eta1 = 0.367  # Efficiency of gas stove
eta2 = 0.424  # Efficiency of electric stove
v_evo = 0.0621  # Evaporation rate (g/s)
S = 408.2  # Steam pot surface area (cm²)
C_o = 1.96  # Specific heat of oil
K_oil = 160  # Oil temperature increase (°C)
m_o = S * 3.5 * 0.9  # Oil mass (g)
e_oven = 2  # Oven power (kW)
e_rice = 0.5  # Rice cooker power (kW)

def microwave(time_min, co2_factor):
    return e_oven * (time_min / 60) * co2_factor * 1.4

def stew(cooking_time_min):
    m_w = 500
    delta_T = 100 - K_water_room_temp
    energy1 = ((m_w / 1000) * C_w * delta_T) / eta1
    dt = cooking_time_min * 60
    evo_kg_s = (v_evo / 1000)
    energy3 = evo_kg_s * S * dt * H_w / eta2
    return (energy1 + energy3) * 1e-6 * mj_to_co2

def boil(cooking_time_min):
    m_w = 1000
    delta_T = 100 - K_water_room_temp
    energy1 = ((m_w / 1000) * C_w * delta_T) / eta1
    dt = cooking_time_min * 60
    evo_kg_s = (v_evo / 1000)
    energy3 = evo_kg_s * S * dt * H_w / eta2
    return (energy1 + energy3) * 1e-6 * mj_to_co2

def steam(cooking_time_min):
    m_w_steam = S * 15 * 0.6
    delta_T = 100 - K_water_room_temp
    energy1 = ((m_w_steam / 1000) * C_w * delta_T) / eta1
    dt = cooking_time_min * 60
    evo_kg_s = (v_evo / 1000)
    energy3 = evo_kg_s * S * dt * H_w / eta2
    return (energy1 + energy3) * 1e-6 * mj_to_co2

def deepfry(cooking_time_min):
    energy_oil = (m_o * C_o * K_oil) / eta1
    return energy_oil * 1e-6 * mj_to_co2

def fry(cooking_time_min):
    energy = (500 * C_o * K_oil) / eta1
    return energy * 1e-6 * mj_to_co2

def oven(time_min, co2_factor):
    return e_oven * (time_min / 60) * co2_factor

def rice_cooker(co2_factor):
    return e_rice * co2_factor

def stir_fry(cooking_time_min):
    energy = (300 * C_o * K_oil) / eta1
    return energy * 1e-6 * mj_to_co2

# ----------------------------
# Step 5. Map original cooking method names to standardized ones
# ----------------------------
method_mapping = {
    "simmer": "simmer",
    "oven": "oven",
    "boil": "boil",
    "steam": "steam",
    "fry": "deepfry",
    "grill": "oven",
    "stir_fry": "stir_fry",
    "sauté": "stir_fry",
    "microwave": "microwave",
    "rice_cooker": "rice_cooker",
}

def parse_cooking_methods(row):
    ct = row.get('cooking_type_ds', None)
    if pd.isna(ct):
        return []
    methods = [m.strip() for m in ct.split(',')]
    mapped_methods = []
    for m in methods:
        m_lower = m.lower()
        if m_lower in ["others", "raw"]:
            continue
        if m_lower == "saute":
            m_lower = "sauté"
        if m_lower in method_mapping:
            mapped_methods.append(method_mapping[m_lower])
    return mapped_methods

df_method['calc_methods'] = df_method.apply(parse_cooking_methods, axis=1)

# ----------------------------
# Step 6. Calculate GHG emissions for each recipe
# ----------------------------
ghg_results = []
for idx, row in df_method.iterrows():
    ghg_total = 0
    methods = row['calc_methods']
    total_time = row['cookingtime']
    country = row.get('country', 'default')
    co2_factor = get_kWh_to_co2(country)

    if not methods:
        ghg_results.append(0)
        continue

    time_each = total_time / len(methods)

    for m in methods:
        if m == 'simmer':
            ghg_total += stew(time_each)
        elif m == 'boil':
            ghg_total += boil(time_each)
        elif m == 'steam':
            ghg_total += steam(time_each)
        elif m == 'deepfry':
            ghg_total += deepfry(time_each)
        elif m == 'fry':
            ghg_total += fry(time_each)
        elif m == 'microwave':
            ghg_total += microwave(time_each, co2_factor)
        elif m == 'oven':
            ghg_total += oven(time_each, co2_factor)
        elif m == 'rice_cooker':
            ghg_total += rice_cooker(co2_factor)
        elif m == 'stir_fry':
            ghg_total += stir_fry(time_each)
    ghg_results.append(ghg_total)

df_method['GHG_cooking_gCO2'] = ghg_results

# Calculate GHG per serving
df_method['GHG_cooking_per_serving_gCO2'] = df_method['GHG_cooking_gCO2'] / df_method['servings_y']

# Before saving, convert index to a regular column
df_method = df_method.reset_index()  # Make 'index' a regular column

# Save & export (avoid saving index again)
df_method.to_excel("/Users/liqiao/Desktop/UTokyo/Food/2024_recipes/Asia_food/2025_data_check/data/20250410/recipes_GHG_cooking_equipment_by_cooking_type_ds.xlsx", index=False)

# Print a preview of key columns
print(df_method[['recipename', 'country', 'cooking_type_ds', 'calc_methods', 'cookingtime', 'servings_y', 
                 'GHG_cooking_gCO2', 'GHG_cooking_per_serving_gCO2']].head())

# Print average emissions
avg_emission = df_method['GHG_cooking_gCO2'].mean()
avg_emission_per_serving = df_method['GHG_cooking_per_serving_gCO2'].mean()
print(f"\nAverage equipment-only cooking GHG emission: {avg_emission:.2f} g CO2 per recipe")
print(f"Average equipment-only cooking GHG emission per serving: {avg_emission_per_serving:.2f} g CO2 per serving")

                                  recipename             country  \
0                                     Afelia              Cyprus   
1  Briam-stuffed Halloumi wrapped in Lountza              Cyprus   
2                                       Aush          Tajikistan   
3              Broiled Beef Rib (Sokalbigui)  Korea, Republic of   
4                                  Bulgogi I  Korea, Republic of   

     cooking_type_ds             calc_methods  cookingtime  servings_y  \
0       Simmer, Oven           [simmer, oven]           40         4.0   
1                Fry                [deepfry]           10         4.0   
2  Simmer, Boil, Fry  [simmer, boil, deepfry]           30         6.0   
3              Grill                   [oven]           30        12.0   
4              Grill                   [oven]           10         6.0   

   GHG_cooking_gCO2  GHG_cooking_per_serving_gCO2  
0        404.072400                    101.018100  
1         54.790326                     13